In [122]:
# Example code for creating a figure of suitable size
# for inclusion in the term paper.

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pandasql as ps
import matplotlib as mpl


plt.rcParams['axes.titlesize'] = 15
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['text.usetex'] = False
# Color cycle for color blind Source: https://gist.github.com/thriveth/8560036
CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']   # https://gist.github.com/thriveth/8560036

# Set the default color cycle
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=CB_color_cycle) 

In [123]:
# Function for a consistent length of figures of 84 mm, converting to inches
def new_figure(height=55):
    "Return figure with width 84mm and given height in mm."

    return plt.figure(figsize=(84/10.16, height/10.16))

# x = range(10)
# y = [v**2 - 1 for v in x]

# fig = new_figure()
# ax = fig.add_subplot(1, 1, 1)
# ax.plot(x, y, 'o-', label='$y=x^2-1$')
# ax.legend()
# ax.set_xlabel('$x$')
# ax.set_ylabel('$y$')
# fig.savefig('sample_plot.pdf', bbox_inches='tight')
# plt.show()

### Reading all the different CSVs produced by each run of the time_it function 

In [124]:
# merge_sort_already_sorted= pd.read_csv("merge_sort_already_sorted.csv")
# merge_sort_random= pd.read_csv("merge_sort_random.csv")
# merge_sort_reverse_sorted= pd.read_csv("merge_sort_reverse_sorted.csv")

# quick_sort_random= pd.read_csv("quick_sort_random.csv")
# quick_sort_already_sorted_till_2560= pd.read_csv("quick_sort_sorted.csv")
# quick_sort_reverse_sorted_till_2560= pd.read_csv("quick_sort_reverse_sorted.csv")

# heap_sort_already_sorted= pd.read_csv("heap_sort_already_sorted.csv")
# heap_sort_random= pd.read_csv("heap_sort_random.csv")
# heap_sort_reverse_sorted= pd.read_csv("heap_sort_reverse_sorted.csv")

# numpy_sort_already_sorted= pd.read_csv("numpy_sort_already_sorted.csv")
# numpy_sort_random= pd.read_csv("numpy_sort_random.csv")
# numpy_sort_reverse_sorted= pd.read_csv("numpy_sort_reverse_sorted.csv")

# python_sorted_already_sorted= pd.read_csv("sorted_already_sorted.csv")
# python_sorted_random= pd.read_csv("sorted_random.csv")
# python_sorted_reverse_sorted= pd.read_csv("sorted_reverse_sorted.csv")         


In [125]:
# dataset=pd.concat([merge_sort_already_sorted, merge_sort_random, merge_sort_reverse_sorted, 
#                    quick_sort_random,quick_sort_already_sorted_till_2560,quick_sort_reverse_sorted_till_2560,
#                    heap_sort_already_sorted, heap_sort_random, heap_sort_reverse_sorted,
#                    numpy_sort_already_sorted, numpy_sort_random, numpy_sort_reverse_sorted,
#                    python_sorted_already_sorted, python_sorted_random, python_sorted_reverse_sorted])


# dataset.loc[ dataset['Data_Type_or_List_type']=='already_sorted', ['Data_Type_or_List_type'] ] ='sorted'
# dataset.loc[ dataset['Data_Type_or_List_type']=='reverese_sorted', ['Data_Type_or_List_type'] ] ='reverse_sorted'
# dataset.loc[ dataset['Data_Type_or_List_type']=='reverse', ['Data_Type_or_List_type'] ] ='reverse_sorted'
# dataset.reset_index(drop=True)

# dataset.to_csv("dataset_concat.csv", index=False)

dataset = pd.read_csv("dataset_concat.csv")
print (np.unique(dataset['Data_Type_or_List_type']).tolist())
print(dataset.shape)

['random', 'reverse_sorted', 'sorted']
(2037, 6)


In [126]:
dataset.head()

,Sort_Type,Data_Type_or_List_type,List_length,Runtimes,Number_of_repeatitions,Datetime
0,merge_sort,sorted,10,0.267837,7,2019-11-01 21:51:45.110982
1,merge_sort,sorted,10,0.277732,7,2019-11-01 21:51:45.110982
2,merge_sort,sorted,10,0.274984,7,2019-11-01 21:51:45.110982
3,merge_sort,sorted,10,0.250366,7,2019-11-01 21:51:45.110982
4,merge_sort,sorted,10,0.233188,7,2019-11-01 21:51:45.110982


In [127]:
# Extract the keys and values and plot them

min_query = """SELECT Sort_Type, 
Data_Type_or_List_type,
List_length, 
min( Runtimes/Number_of_repeatitions)  as Single_runtime
FROM dataset 
GROUP BY Sort_Type, 
Data_Type_or_List_type,
List_length """

df_min = pd.DataFrame( ps.sqldf(min_query) )


print (np.unique(df_min['Data_Type_or_List_type']).tolist())
df_min.head()


['random', 'reverse_sorted', 'sorted']


,Sort_Type,Data_Type_or_List_type,List_length,Single_runtime
0,heap_sort,random,10,0.025611
1,heap_sort,random,20,0.024896
2,heap_sort,random,40,0.029855
3,heap_sort,random,80,0.042957
4,heap_sort,random,160,0.049812


In [128]:
df_min['Single_runtime'] = df_min['Single_runtime']*1000000000

In [129]:
df_min.head()

,Sort_Type,Data_Type_or_List_type,List_length,Single_runtime
0,heap_sort,random,10,2.561117e+07
1,heap_sort,random,20,2.489582e+07
2,heap_sort,random,40,2.985538e+07
3,heap_sort,random,80,4.295699e+07
4,heap_sort,random,160,4.981248e+07


In [159]:
# Prove that the graph in NlogN
# Changing the constants by Hit and Trial
# The time in plotted in microseconds
def plot_minimum_times(input_type='sorted', lower_limit=81920, upper_limit=10485760):
    filter01 =  (df_min['Data_Type_or_List_type']==input_type)
    plot_data = df_min[filter01]

    filter02 = (plot_data['List_length']<=upper_limit) &  (plot_data['List_length']>=lower_limit)
    plot_data = plot_data[filter02]

    fig = new_figure()
    list_of_sorts= sorted( np.unique(plot_data['Sort_Type']).tolist() , reverse=True)
    list_of_sorts = [  'numpy_sort', 'python_sorted']
    for sort_type in list_of_sorts:
        filter03 = (plot_data['Sort_Type']==sort_type)
        plot_this = plot_data[filter03]   
        
        size_n = plot_this['List_length']
        time_y = plot_this['Single_runtime']
        
        size_n_norm = ( size_n - np.mean(size_n) )/ np.std(size_n)
        time_y_norm = ( time_y - np.mean(time_y) )/ np.std(time_y)        
        plt.plot( size_n_norm , time_y_norm ,'-o' ,alpha=0.8, label=sort_type)
    plt.plot( [-1,0,2.5], [-1 ,0,2.5] ,'-',color='black' )
    plt.vlines(x=0,ymin=-1.5, ymax=2.5,linestyles= '--', color='black')
    plt.hlines(y=0,xmin=-1.5, xmax=2.5,linestyles= '--', color='black')
                 
    plt.xlabel('Normalized size of the numeric array')
    plt.ylabel('Normalized Time')
    plt.title("Plot against linear line -"+input_type+" data")
    plt.legend()
    plt.tight_layout()
    plt.savefig("normalised\Check against linear runtime.pdf" , bbox_inches='tight')
    plt.show()

## Check against linear runtime- Python Sorted and Numpy Sort

In [160]:
from ipywidgets import interact #,interactive , fixed, interact_manual
# import ipywidgets as widgets
pickle_object= interact(plot_minimum_times, input_type=['random', 'reverse_sorted', 'sorted']
         ,lower_limit=  np.unique(df_min['List_length']).tolist() 
         ,upper_limit=  np.unique(df_min['List_length']).tolist()
        )

interactive(children=(Dropdown(description='input_type', index=2, options=('random', 'reverse_sorted', 'sorted…

In [157]:
# Changing the constants by Hit and Trial
# The time in plotted in microseconds
x =  np.linspace(9,999,9999)
y = x*sqrt(x)
x_norm = ( x - np.mean(x) )/ np.std(x)
y_norm = ( y - np.mean(y) )/ np.std(y) 
# x_norm= x_norm[5000:]
# y_norm= y_norm[5000:]


def plot_minimum_times(input_type='sorted', lower_limit=81920, upper_limit=10485760):
    filter01 =  (df_min['Data_Type_or_List_type']==input_type)
    plot_data = df_min[filter01]

    filter02 = (plot_data['List_length']<=upper_limit) &  (plot_data['List_length']>=lower_limit)
    plot_data = plot_data[filter02]

    fig = new_figure()
    list_of_sorts= sorted( np.unique(plot_data['Sort_Type']).tolist() , reverse=True)
    list_of_sorts = [  'heap_sort', ] #'merge_sort']
    for sort_type in list_of_sorts:
        filter03 = (plot_data['Sort_Type']==sort_type)
        plot_this = plot_data[filter03]   
        
        size_n = plot_this['List_length']
        time_y = plot_this['Single_runtime']
        
        size_n_norm = ( size_n - np.mean(size_n) )/ np.std(size_n)
        time_y_norm = ( time_y - np.mean(time_y) )/ np.std(time_y)        
        plt.plot( size_n_norm , time_y_norm ,'-o' ,alpha=0.8, label=sort_type)
        
        plt.vlines(x=0,ymin=-0.5, ymax=2.5,linestyles= '--', color='black')
        plt.hlines(y=0,xmin=-0.5, xmax=2.5,linestyles= '--', color='black')
        #plot n*sqrt(n)
        
    plt.plot( [-1,0,2.5], [-1 ,0,2.5] ,'--' ,color='black', label='linear')  
    plt.plot( x_norm, y_norm  ,'-', label='$n*sqrt(n)$' )
    
    plt.xlabel('Normalized size of the numeric array')
    plt.ylabel('Normalized Time')
    plt.title("Runtimes of sort Algorithms for -"+input_type+" data")
    plt.legend()
    plt.tight_layout()
#     plt.savefig("normalised\Check against n*sqrt(n) runtime.pdf" , bbox_inches='tight')
    plt.show()

NameError: name 'sqrt' is not defined

In [158]:
from ipywidgets import interact #,interactive , fixed, interact_manual
# import ipywidgets as widgets
pickle_object= interact(plot_minimum_times, input_type=['random', 'reverse_sorted', 'sorted']
         ,lower_limit=  np.unique(df_min['List_length']).tolist() 
         ,upper_limit=  np.unique(df_min['List_length']).tolist()
        )

interactive(children=(Dropdown(description='input_type', index=2, options=('random', 'reverse_sorted', 'sorted…

In [140]:
x =  np.linspace(0,10,9999)
y = x*np.sqrt(x)
x_norm = ( x - np.mean(x) )/ np.std(x)
y_norm = ( y - np.mean(y) )/ np.std(y) 
x_norm= x_norm[5000:]
y_norm= y_norm[5000:]

In [162]:
import ipywidgets
ipywidgets.__version__

'7.5.1'